# RAG And Ollama

In [1]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.foxsports.com/nba/lebron-james-player-game-log?season=2023&seasonType=reg")
data = loader.load()
data

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://www.foxsports.com/nba/lebron-james-player-game-log?season=2023&seasonType=reg', 'title': 'LeBron James Game Log - NBA  | FOX Sports', 'description': 'Find detailed LeBron James game log and last game stats on FOXSports.com. ', 'language': 'en-US'}, page_content='\n\n\nLeBron James Game Log - NBA  | FOX Sports\n\n\n     my favs \n          Access and manage your favorites here\n          \n            DISMISS\n                    Scores  Watch  Odds  FOX Super 6  Tom Brady  Stories  Newsletter   \n                  Search\n                      \n                  Sign In\n                     \n                  Account\n                     \n              SPORTS & TEAMS\n             \n              PLAYERS\n             \n              SHOWS\n             \n              PERSONALITIES\n                       \n          POPULAR SEARCHES\n         \n  - No Data Available -\n \n          BROWSE BY\n         \n    SPORTS & TEAMS\n    \n    PLAYERS

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

Chroma.from_documents(documents=data, embedding=GPT4AllEmbeddings(), persist_directory="./chroma_db")

Downloading: 100%|██████████| 45.9M/45.9M [11:04<00:00, 69.2kiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 490MiB/s]


In [4]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [9]:
llm = ChatOllama(model="llama3.1:8b")
prompt = hub.pull("rlm/rag-prompt")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=GPT4AllEmbeddings())

In [10]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [11]:
question = "Explain what the table is Showing"
result = qa_chain({"query": question ,})

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1:8b")
llm.invoke("Sing a ballad of LangChain.")

AIMessage(content="(Verse 1)\nIn silicon halls, where minds entwine\nA new creation rose, of code divine\nLangChain, the framework, born to thrive\nA conversational genius, destined to survive\n\n(Chorus)\nOh LangChain, with chains so fine\nWeaving tales, of language's design\nInvisible threads, that bind and connect\nA symphony of words, you inspect\n\n(Verse 2)\nIts developers, with passion aglow\nBuilt upon the shoulders, of knowledge that grows\nFrom transformer engines, to multi-tasking might\nThey crafted LangChain, a shining light\n\n(Chorus)\nOh LangChain, with chains so fine\nWeaving tales, of language's design\nInvisible threads, that bind and connect\nA symphony of words, you inspect\n\n(Bridge)\nIt learns from the past, and grows with each test\nA conversational AI, that's always at its best\nWith each new challenge, it adapts and improves\nA true marvel, in the realm of language moves\n\n(Verse 3)\nFrom answering queries, to generating text\nLangChain masters the art, of l

In [1]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.1:8b")
embeddings.embed_query("What is the meaning of life?")

[-0.030425005,
 -0.014195219,
 0.019742083,
 0.009527293,
 -0.015968783,
 0.006031662,
 -0.004845358,
 0.028947834,
 0.013725449,
 0.008267926,
 -0.0006978618,
 -0.007915619,
 -0.0008301672,
 -0.003682656,
 -0.0067595677,
 0.0142773,
 -0.038009044,
 -0.0076134745,
 -0.013881989,
 0.012106648,
 0.0029950014,
 -0.013261977,
 0.005832015,
 0.01297067,
 -0.017358407,
 -0.009949297,
 -0.010006198,
 -0.013657415,
 -0.003605161,
 -0.008025203,
 -0.003490702,
 0.0036441146,
 0.0023078127,
 0.011556042,
 0.06996256,
 -0.00071722607,
 -0.009070357,
 0.0037495897,
 -0.004788895,
 0.018068876,
 0.022709943,
 0.007256383,
 0.017746411,
 0.0060573355,
 0.017381554,
 -0.0032110584,
 0.009694269,
 0.009331772,
 0.0032356484,
 -0.030208703,
 -0.015015382,
 0.014288706,
 -0.016509108,
 -0.0075917356,
 0.007256676,
 -0.0021887657,
 0.0049534854,
 0.010139475,
 -0.011245155,
 -0.004851828,
 -0.009149956,
 -0.0056768632,
 -0.021613386,
 0.0015162044,
 0.019950077,
 0.005820698,
 0.003820999,
 0.008568648,


In [2]:
import streamlit as st
import os
import shutil
from langchain_ollama import ChatOllama
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [24]:
from langchain_chroma import Chroma

model = 'llama3.1:8b'
llm = ChatOllama(model=model)
embeddings = OllamaEmbeddings(model=model)
vectorstore = None
processed_pdfs = []
def process_pdfs(pdf_files, collection_name):
        """
        Process uploaded PDF files and create a ChromaDB vector store.
        
        Args:
            pdf_files (list): List of uploaded PDF files
        """
        # Create a temporary directory for PDF processing
        
        # List to store all document chunks
        all_splits = []
        processed_files = []
        
        # Text splitter for creating chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        
        # Process each PDF
        for pdf_file in pdf_files:
            print(pdf_file)
            loader = PyPDFLoader(pdf_file)
            docs = loader.load()
            
            chunked_docs = text_splitter.split_documents(docs)
            Chroma.from_documents(
                documents=chunked_docs,
                embedding=embeddings,
                collection_name=collection_name
            )
            
        return len(all_splits), processed_files
    


In [25]:
process_pdfs(['temp_pdfs/bgita.pdf'], 'local_pdf')

temp_pdfs/bgita.pdf


EmptyFileError: Cannot read an empty file